# Import backage

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from train import train_plus
from models.models_new import Renorm_Dynamic
from dynamic_models_sis_new import Simple_Spring_Model
from vector_field import mesh
from vector_field import transit
from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Generate data

In [3]:
mul_batch_size = [0,5000,3000,1000]#[0,10,4500,1500]
sigma = 0.03
rou = -0.5
steps = 7
dt = 0.01
interval = 100
steps2 = 10
seed = 2050
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
spring = Simple_Spring_Model(device=device)
test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10*interval, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt,interval=interval) 
train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps*interval, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=dt,interval=interval)

# Train

In [4]:
sz = 4
scale = 2
L = 1
mae2_w = 3
T_total = 30001
eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train_plus(train_data=train_data, test_data=test_data, 
                                                                         sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                         mae2_w=mae2_w, dt=dt, T2=T_total,framework='nis',steps2=steps2,interval=interval)
torch.save(net.state_dict(), f'mdl_data/nis steps10 exp-0.2t interval100 seed={seed}.mdl')

check point-------o_0-> 10:52:09.657308; lasting 0:00:09.943540 seconds
Epoch: 0
Train loss: 0.2784
dEI: 0.4863
term1: -0.4070
term2: 1.3796
Test multistep loss: 0.5502
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 10:52:16.575406; lasting 0:00:06.918098 seconds
check point-------o_0-> 10:54:25.181453; lasting 0:02:08.606047 seconds
Epoch: 500
Train loss: 0.0267
dEI: 1.1884
term1: 0.9884
term2: 1.3884
Test multistep loss: 0.2355
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 10:54:36.430977; lasting 0:00:11.249524 seconds
check point-------o_0-> 10:56:35.675362; lasting 0:01:59.244385 seconds
Epoch: 1000
Train loss: 0.0192
dEI: 1.4103
term1: 1.4461
term2: 1.3745
Test multistep loss: 0.2327
--------------------------------------------------------------------------------------------------------

In [5]:
mul_batch_size = [0,5000,3000,1000]#[0,10,4500,1500]
sigma = 0.03
rou = -0.5
steps = 7
dt = 0.01
interval = 100
steps2 = 10

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
spring = Simple_Spring_Model(device=device)
test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10*interval, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt,interval=interval) 
train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps*interval, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=dt,interval=interval)

In [ ]:
eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net_nisp = train_plus(train_data=train_data, test_data=test_data, 
                                                                         sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                         mae2_w=mae2_w, dt=dt, T2=T_total,framework='nis+',steps2=steps2,interval=interval)
torch.save(net_nisp.state_dict(), f'mdl_data/nisp steps10 exp-0.2t interval100 seed={seed}.mdl')

check point-------o_0-> 13:05:40.201373; lasting 0:00:11.394018 seconds
Epoch: 0
Train loss: 0.2784
dEI: 0.4863
term1: -0.4070
term2: 1.3796
Test multistep loss: 0.5546
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 13:05:48.021163; lasting 0:00:07.819790 seconds
check point-------o_0-> 13:08:18.401984; lasting 0:02:30.380821 seconds
Epoch: 500
Train loss: 0.0264
dEI: 1.1925
term1: 0.9970
term2: 1.3881
Test multistep loss: 0.2403
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 13:08:28.045747; lasting 0:00:09.643763 seconds
check point-------w_0-> 13:10:57.811657; lasting 0:02:29.765910 seconds
check point-------w_1-> 13:10:57.840666; lasting 0:00:00.029009 seconds
check point-------w_2-> 13:15:58.621614; lasting 0:05:00.780948 seconds
check point-------w_3-> 13:15:58.635687; lasting 0:00:00.01

# Plot

In [ ]:
from vector_field import vector_func
sz = 4
scale = 2
L = 1
sigma = 0.03
rou = -0.5
hidden_units = 64
nis = Renorm_Dynamic(sym_size = sz, latent_size = scale, effect_size = sz, 
                         hidden_units = hidden_units, normalized_state=True, device = device)
nisp = Renorm_Dynamic(sym_size = sz, latent_size = scale, effect_size = sz, 
                         hidden_units = hidden_units, normalized_state=True, device = device)
nis.to(device=device)
nisp.to(device=device)
nis.load_state_dict(torch.load(f'mdl_data/nis steps10 exp-0.2t interval100 seed={seed}.mdl'))
nisp.load_state_dict(torch.load(f'mdl_data/nisp steps10 exp-0.2t interval100 seed={seed}.mdl'))



dd1_nis,dd2_nis=vector_func(nis,sigma=sigma,rou=rou, jac_bool=False, density=0.007,density2=0.015,density3=0.07)
dd1_nis_jac,dd2_nis_jac=vector_func(nis,sigma=sigma,rou=rou, jac_bool=True,density=0.007,density2=0.015,density3=0.07)
dd1_nisp,dd2_nisp=vector_func(nisp,sigma=sigma,rou=rou, jac_bool=False,density=0.007,density2=0.015,density3=0.07)
dd1_nisp_jac,dd2_nisp_jac=vector_func(nisp,sigma=sigma,rou=rou, jac_bool=True,density=0.007,density2=0.015,density3=0.07)

In [ ]:
import matplotlib
def SIR(s,i,lam=1,miu=0.5):
    ds=-lam * s * i
    di=lam * s * i - miu * i
    return ds, di
S, I = mesh(0.07)
dSdt,dIdt = SIR(S,I)
m=0.1
dSdt,dIdt=m*dSdt,m*dIdt
fig,ax = plt.subplots(figsize=(5,4),dpi=150)
colorlabel = ['#F4F1DE','#DF7A5E','#3C405B','#82B29A','#F2CC8E']
colors=[colorlabel[2]]*len(S)+[colorlabel[3]]*len(S)+[colorlabel[1]]*len(S)
S=list(S)+list(S)+list(S)
I=list(I)+list(I)+list(I)
dd1_final=list(dd1_nis)+list(dd1_nis_jac)+list(dSdt)
dd2_final=list(dd2_nis)+list(dd2_nis_jac)+list(dIdt)

quiver = ax.quiver(S, I, dd1_final, dd2_final, width=0.005,color=colors)
red_patch = matplotlib.patches.Patch(color=colorlabel[1], label='real')
green_patch = matplotlib.patches.Patch(color=colorlabel[2], label='NIS')
ran_patch = matplotlib.patches.Patch(color=colorlabel[3], label='theoretic')
plt.legend(handles=[red_patch, green_patch,ran_patch],loc='best', fontsize=12)

# Set graphic title and axis labels.
plt.xlabel('$S(h_1)$',fontsize=15)
plt.ylabel('$I(h_2)$',fontsize=15)

ax.tick_params(labelsize=15)
plt.show()

In [ ]:
def SIR(s,i,lam=1,miu=0.5):
    ds=-lam * s * i
    di=lam * s * i - miu * i
    return ds, di
S, I = mesh(0.07)
dSdt,dIdt = SIR(S,I)
m=0.1
dSdt,dIdt=m*dSdt,m*dIdt
fig,ax = plt.subplots(figsize=(5,4),dpi=150)
colorlabel = ['#F4F1DE','#DF7A5E','#3C405B','#82B29A','#F2CC8E']
colors=[colorlabel[2]]*len(S)+[colorlabel[3]]*len(S)+[colorlabel[1]]*len(S)
S=list(S)+list(S)+list(S)
I=list(I)+list(I)+list(I)
dd1_final=list(dd1_nisp)+list(dd1_nisp_jac)+list(dSdt)
dd2_final=list(dd2_nisp)+list(dd2_nisp_jac)+list(dIdt)

quiver = ax.quiver(S, I, dd1_final, dd2_final, width=0.005,color=colors)
red_patch = matplotlib.patches.Patch(color=colorlabel[1], label='real')
green_patch = matplotlib.patches.Patch(color=colorlabel[2], label='NIS+')
ran_patch = matplotlib.patches.Patch(color=colorlabel[3], label='theoretic')
plt.legend(handles=[red_patch, green_patch,ran_patch],loc='best', fontsize=12)

# Set graphic title and axis labels.
plt.xlabel('$S(h_1)$',fontsize=15)
plt.ylabel('$I(h_2)$',fontsize=15)

ax.tick_params(labelsize=15)
plt.show()

In [ ]:
seed